# RED NEURONAL DE UN CONJUNTO DE ENFERMEDADES

# IMPORTAR LAS LIBRERIAS

In [2]:
# tensorflow
import tensorflow
print('tensorflow: %s' % tensorflow.__version__)
# keras
import keras
print('keras: %s' % keras.__version__)

import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

/home/hexlinux/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hexlinux/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hexlinux/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hexlinux/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
# CARGAR LAS IMAGES 

In [6]:
from skimage.transform import resize

dirname = os.path.join(os.getcwd(), 'Cultivos de hortalizas')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            
            image_resized = resize(image, (21, 28),anti_aliasing=True,clip=False,preserve_range=True)
            images.append(image_resized)
            
    
            #images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Apio/Tizon 1
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Cebolla/Mancha purpura 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Cebolla/Moho gris 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Chile/Cercosporiosis 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Esparrago/Mancha foliar 12
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Fresa/Mancha foliar 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Mani/Cercosporiosis 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Melon/Gomosis 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Papa/Rona 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Pepino/Mancha Angular 11
/media/hexlinux/FILE/P - Tus Cultivos./Cultivos de hortalizas/Rabano/Roya Blanca 11


# CREAR ETIQUETAS Y CLASES

In [7]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

enfermedades=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    enfermedades.append(name[len(name)-1])
    indice=indice+1
    
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)  

Cantidad etiquetas creadas:  155
0 Tizon
1 Mancha purpura
2 Moho gris
3 Cercosporiosis
4 Mancha foliar
5 Mancha foliar
6 Cercosporiosis
7 Gomosis
8 Rona
9 Mancha Angular
10 Roya Blanca
11 Cabeza negra (pudricion)
12 Marchites Vascular
13 Alternaria


ValueError: could not broadcast input array from shape (21,28,3) into shape (21,28)

# CREAMOS CONJUNTO DE ENTRENAMIENTO Y VALIDACIÓN

In [ ]:
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(train_Y[0]))

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(test_Y[0]))

# PREPROCESAMOS LAS IMAGENES

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

#  HACEMOS LA CODIFICACIÓN ONE-HOT PARA LA RED

In [ ]:
# Cambie las etiquetas de codificación categórica a one-hot
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Mostrar el cambio de etiqueta de categoría usando codificación one-hot
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

# CREAMOS EL CONJUNTO DE ENTRENAMIENTO Y VALIDACIÓN  

In [ ]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [ ]:
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

# CREAMOS EL MODELO CNN 

In [ ]:
#declaramos variables con los parámetros de configuración de la red
INIT_LR = 1e-3 # Valor inicial de learning rate. El valor 1e-3 corresponde con 0.001
epochs = 100 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 100 # cantidad de imágenes que se toman a la vez en memoria

In [ ]:
sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))

sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5))
sport_model.add(Dense(nClasses, activation='softmax'))

In [ ]:
sport_model.summary()

In [ ]:
sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

# ENTRENAMOS EL MODELO PARA QUE APRENDA A CLASIFICAR LAS      IMAGENES

In [ ]:
sport_train = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=2,validation_data=(valid_X, valid_label))

# GUARDAMOS EL MODELO

In [ ]:
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar  (verbose estaba en 1)
sport_model.save("plantasprueba.h5py")

# EVALUAMOS LA RED NEURONAL

In [ ]:
test_eval = sport_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
history_dict = sport_train.history
print(history_dict.keys())

In [ ]:
accuracy = sport_train.history['accuracy']
val_accuracy = sport_train.history['val_accuracy']
loss = sport_train.history['loss']
val_loss = sport_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes2 = sport_model.predict(test_X)

In [ ]:
predicted_classes=[]
for predicted_sport in predicted_classes2:
    predicted_classes.append(predicted_sport.tolist().index(max(predicted_sport)))
predicted_classes=np.array(predicted_classes)

In [ ]:
predicted_classes.shape, test_Y.shape

# LA RED APRENDE A MEJOR DE LOS ERRORES

In [ ]:
correct = np.where(predicted_classes==test_Y)[0]
print("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(21,28,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(deportes[predicted_classes[correct]],
                                                    deportes[test_Y[correct]]))

    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_classes!=test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(21,28,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(deportes[predicted_classes[incorrect]],
                                                    deportes[test_Y[incorrect]]))
    plt.tight_layout()

In [ ]:
target_names = ["Class {}".format(i) for i in range(nClasses)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

# PREDECIR LAS IMAGENES

In [ ]:
from skimage.transform import resize

images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
filenames = ['22.jpg']

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (21, 28),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = sport_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    plt.imshow(image)
    plt.show()
    print(filenames[i], enfermedades[img_tagged.tolist().index(max(img_tagged))])